# Spike sorting on RTX 5090

10 min recording |

author: laquitainesteeve@gmail.com

Tested on an Ubuntu 24 with a 32GB VRAM Nvidia RTX 5090

Execution time: 16 min

## Setup 

1. Enable forward compatibility if your GPU and CUDA libraries are more recent and not supported by editing your matlab `startup.m` file to contain "parallel.gpu.enableCUDAForwardCompatibility(true)" and:

    ```bash
    # manually compile Kilosort3 with CUDA support for forward compatibility
    sudo apt install gcc-11 g++-11 # install gcc 11 compiler
    sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 100 # enable temporary
    cd /home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp/CUDA/
    matlab -batch mexGPUall  # compile matlab mex files
    ```

2. Activate `spikesort_rtx5090` environment and select kernel

    ```bash
    python -m ipykernel install --user --name spikesort_rtx5090 --display-name "spikesort_rtx5090"
    ```

3. Run notebook.



In [19]:
%%time 
%load_ext autoreload
%autoreload 2

# import python packages
import os
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface as si
print("spikeinterface", si.__version__)

# project path
PROJ_PATH = "/home/steeve/steeve/epfl/code/spikebias/"
os.chdir(PROJ_PATH)

# import spikebias package
from src.nodes.sorting import sort_and_postprocess_10m

# recording parameters
REC_SECS = 600 
RECORDING_PATH = "./dataset/00_raw/recording_npx_spont/"

# sorting parameters
SORTER = "kilosort3"
SORTER_PATH = "/home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp"
SORTING_OUTPUT = "./temp/SortingKS3/" 
SORTER_PARAMS = {
    "detect_threshold": 6,
    "projection_threshold": [9, 9],
    "preclust_threshold": 8,
    "car": True,
    "minFR": 0, # modified
    "minfr_goodchannels": 0, # modified
    "nblocks": 5,
    "sig": 20,
    "freq_min": 300,
    "sigmaMask": 30,
    "lam": 20.0,
    "nPCs": 3,
    "ntbuff": 64,
    "nfilt_factor": 4,
    "do_correction": True,
    "NT": 65792, # modified    
    "AUCsplit": 0.8,
    "wave_length": 61,
    "keep_good_only": False,
    "skip_kilosort_preprocessing": False,
    "scaleproc": None,
    "save_rez_to_mat": False,
    "delete_tmp_files": ("matlab_files",),
    "delete_recording_dat": False,
}

# SET KS3 software environment variable
ss.Kilosort3Sorter.set_kilosort3_path(SORTER_PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
spikeinterface 0.100.5
Setting KILOSORT3_PATH environment variable for subprocess calls to: /home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp
CPU times: user 382 µs, sys: 0 ns, total: 382 µs
Wall time: 363 µs


## npx spont

In [ ]:
# setup configuration
CFG = {
    'probe_wiring': {
        'full': {
            'output': 'dataset/00_raw/recording_npx_spont'
        }
    },
    'preprocessing': {
        'full': {
            'output': {
                'trace_file_path': 'dataset/01_intermediate/preprocessing/recording_npx_spont'
            }
        }
    },
    'sorting': {
        'sorters': {
            f"{SORTER}": {
                '10m': {
                    'output': './temp/SortingKS3_10m_RTX5090', #'path/to/sorting/output',
                    'sort_output':'./temp/KS3_output/KS3_output_10m_RTX5090' #'path/to/sorting/sort_output'
                }
            }
        }
    },
    'postprocessing': {
        'waveform': {
            'sorted': {
                'study': {
                    f"{SORTER}": {  # sorter name
                        '10m': './temp/study_ks3_10m_RTX5090' #'path/to/postprocessing/study'
                    }
                }
            }
        }
    }
}

# spike sort
sort_and_postprocess_10m(CFG, SORTER, SORTER_PARAMS, duration_sec=REC_SECS,
                         is_sort=True, is_postpro=False, extract_wvf=False, copy_binary_recording=True,
                         remove_bad_channels=True)

# post-process
sort_and_postprocess_10m(CFG, SORTER, SORTER_PARAMS, duration_sec=REC_SECS,
                         is_sort=False, is_postpro=True, extract_wvf=True, copy_binary_recording=True,
                         remove_bad_channels=False)

In [ ]:
# load old (reference) sorting and compare with sorting on RTX5090
SortingRef = si.load_extractor("./dataset/01_intermediate/sorting/npx_spont/SortingKS3_10m")
Sorting = si.load_extractor("./temp/SortingKS3_10m_RTX5090_DriftCorr")

print(len(SortingRef.unit_ids))
print(len(Sorting.unit_ids))

803
723
